In [1]:
if Sys.iswindows()
    username = "Alex"
    pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
    using Pkg
    Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
    using StaticArrays, DifferentialEquations, DynamicalSystems, JLD2, CairoMakie
    include("C:\\Users\\Alex\\Desktop\\dynamical-systems\\system.jl");
else
    username = "nova"
    pathtorepo = "/home/nova/work/repo_ds/dynamical-systems"
    using Pkg
    Pkg.activate(pathtorepo * "/env/integrate/")
    using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie
    include("/home/nova/work/repo_ds/dynamical-systems/system.jl")
end

  Activating project at `~/work/repo_ds/dynamical-systems/env/integrate`


TM6_glial_ECM_help (generic function with 1 method)

In [2]:
function save_logs(index, u0_st, u0_loc)
    open(logs_Λss, "a") do file
        print(file, "index: $(index); αE: $(αE_range[index]); \n");
        print(file, "u0 start: $(u0_st); \n");
        print(file, "u0 end: $(u0_loc) \n");
        print(file, "\n");
    end
end

save_logs (generic function with 1 method)

In [13]:
len = 250;
αE_range = range(0.067, 5.0, length = len);

u0s = zeros(len, 6);
Λss = zeros(len, 6);
logs_Λss = "lse_logs_journal.txt";

t_tt = 3000;
t_lse = 5000;

tol = 1e-15;
integrator_setting = (alg = Vern9(), adaptive = true, abstol = tol, reltol = tol);

index_control_param = 6;

u0start = [0.9445509341100914, 0.74116702856987, 0.7361196042973006, 0.0646914552140727, 0.15145764079879162, 0.0009327645775731449];
params = TM6_glial_ECM_get_params();
ds = CoupledODEs(TM6_glial_ECM, u0start, params, diffeq = integrator_setting);

In [14]:
for index in eachindex(αE_range)

    if index == 1
        global u0_loc = u0start;
    end
    
    u0_st = u0_loc # initial condition without truncate transient time

    set_parameter!(ds, index_control_param, αE_range[index]);
    tr, _ = trajectory(ds, t_tt, u0_loc);

    u0_loc = tr[end]; # initial condition with truncate transient time
    u0s[index, :] = u0_loc;
    save_logs(index, tr[1], tr[end])
end